In [1]:
# Include project root in path
import sys

sys.path.append("../..")
sys.path.append("./magic-ml-images")
sys.path.append("../../magic-ml-images")

# this increases performance?
import torch
torch.multiprocessing.set_start_method("spawn")

# Load data
from src.common import preprocessing, datasets
import pandas as pd

gammas = datasets.read_gammas()
protons = datasets.read_protons()

gammas["class"] = 1.0
protons["class"] = 0.0

data = pd.concat([gammas, protons])

# Preprocess data
from src.common.preprocessing import preprocess
from src.common import (
    PARAMS_HILLAS,
    PARAMS_TRUE_SHOWER,
    PARAMS_IMAGE_M1,
    PARAMS_IMAGE_M2,
    PARAMS_CLEAN_IMAGE_M1,
    PARAMS_CLEAN_IMAGE_M2,
)

train, validation, test = preprocess(
    data,
    normalize_params=PARAMS_HILLAS + PARAMS_CLEAN_IMAGE_M1 + PARAMS_CLEAN_IMAGE_M2,
    stratify_column_name="class",
)

In [2]:
# Since we want to use torch, we now build torch datasets from previous Pandas dataframes
from torch.utils.data import Dataset


class ParticleClassificationDataset(Dataset):
    def __init__(
        self,
        df,
    ):
        self.images = [
            df[PARAMS_CLEAN_IMAGE_M1].values,
            df[PARAMS_CLEAN_IMAGE_M2].values,
        ]
        self.features = df[PARAMS_HILLAS].values
        self.labels = df[["class"]].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        images = torch.stack(list(map(lambda image: torch.tensor(image[idx], dtype=torch.float), self.images))).to(device)
        features = torch.tensor(self.features[idx], dtype=torch.float)
        label = torch.squeeze(torch.tensor(self.labels[idx], dtype=torch.float))
        return images, features, label


train_dataset = ParticleClassificationDataset(train)
validation_dataset = ParticleClassificationDataset(validation)
test_dataset = ParticleClassificationDataset(test)

In [3]:
# Select a supported torch device
import torch
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cuda:0"
print(f"Selected device: {device}")

Selected device: cuda:0


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.common.HexaToParallelogram import HexaToParallelogram


class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(2, 8, 5), # 2, 6, 5
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(8, 16, 5),
            nn.MaxPool2d(2, 2),
        )

    def forward(self, images):
        x = self.model(images)
        return x


class ParticleCNNClassifier(nn.Module):
    def __init__(self, num_additional_parameters):
        super().__init__()

        self.hex2par = HexaToParallelogram(2)
        self.cnn = CNN()

        self.fc1 = nn.Linear(16 * 6 * 6 + num_additional_parameters, 128) 
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, images, additional_parameters):
        x = self.hex2par(images)

        x = self.cnn(x)
        x = torch.flatten(x, 1)

        x = torch.cat((x, additional_parameters), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time

def train_model(model, train_dataset, validation_dataset, optimizer, epochs=10, batch_size=32):
    model.to(device)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    
    criterion = nn.BCEWithLogitsLoss()
    
    for epoch in range(epochs):
        epoch_start = time.time()
        model.train()
        epoch_accum_loss = 0
        correct, total = 0, 0
        
        for images, additional_features, labels in train_loader:
            images, additional_features, labels = images.to(device), additional_features.to(device), labels.to(device).float().unsqueeze(1)

            optimizer.zero_grad()

            outputs = model(images, additional_features)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            epoch_accum_loss += loss.item()
            predicted = torch.round(torch.sigmoid(outputs))
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        epoch_accum_loss /= len(train_loader)
        epoch_accuracy = correct / total
        
        val_loss, val_acc = evaluate_model(model, validation_loader, criterion, device)
        
        epoch_end = time.time()
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {epoch_accum_loss:.4f} Acc: {epoch_accuracy:.4f} | Val Loss: {val_loss:.4f} Val Acc: {val_acc:.4f} | Epoch took {epoch_end - epoch_start:.2f}s")

def evaluate_model(model, dataloader, criterion, device) -> (float, float):
    model.eval()
    loss = 0
    correct, total = 0, 0
    
    with torch.no_grad():
        for images, additional_features, labels in dataloader:
            images, additional_features, labels = images.to(device), additional_features.to(device), labels.to(device).float().unsqueeze(1)

            outputs = model(images, additional_features)
            loss += criterion(outputs, labels).item()
            predicted = torch.round(torch.sigmoid(outputs))
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    return loss / len(dataloader), correct / total


In [6]:
model = ParticleCNNClassifier(len(PARAMS_HILLAS))
model.to(device)

# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.0, weight_decay=0.001)
# lr=0.001, momentum=0.0, weight_decay=0.001
# optimizer = optim.Adam(lr=lr)
optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-3)

train_model(model, train_dataset, validation_dataset, optimizer, epochs=30, batch_size=64)

Epoch 1/30 - Train Loss: 0.4045 Acc: 0.8182 | Val Loss: 0.3658 Val Acc: 0.8478 | Epoch took 75.33s
Epoch 2/30 - Train Loss: 0.3649 Acc: 0.8421 | Val Loss: 0.3571 Val Acc: 0.8472 | Epoch took 75.53s
Epoch 3/30 - Train Loss: 0.3590 Acc: 0.8441 | Val Loss: 0.3671 Val Acc: 0.8419 | Epoch took 75.97s
Epoch 4/30 - Train Loss: 0.7087 Acc: 0.6933 | Val Loss: 0.6466 Val Acc: 0.6513 | Epoch took 75.09s
Epoch 5/30 - Train Loss: 0.6473 Acc: 0.6513 | Val Loss: 0.6469 Val Acc: 0.6513 | Epoch took 76.89s
Epoch 6/30 - Train Loss: 0.5319 Acc: 0.7418 | Val Loss: 0.4340 Val Acc: 0.7997 | Epoch took 74.26s
Epoch 7/30 - Train Loss: 0.3765 Acc: 0.8355 | Val Loss: 0.3775 Val Acc: 0.8375 | Epoch took 76.27s
Epoch 8/30 - Train Loss: 0.3607 Acc: 0.8436 | Val Loss: 0.3534 Val Acc: 0.8461 | Epoch took 75.76s
Epoch 9/30 - Train Loss: 0.3910 Acc: 0.8281 | Val Loss: 0.3970 Val Acc: 0.8256 | Epoch took 75.44s
Epoch 10/30 - Train Loss: 0.3649 Acc: 0.8423 | Val Loss: 0.3525 Val Acc: 0.8485 | Epoch took 74.95s
Epoch 11/